# Building a RAG system with OpenAI and ChromaDB

#### Introduction
Retrieval-Augmented Generation (RAG) is a powerful technique that combines the capabilities of large language models with external knowledge retrieval. This notebook will walk you through building a complete RAG system using:

- LangChain: A framework for developing applications powered by language models
- ChromaDB: An open-source vector database for storing and retrieving embeddings
- OpenAI: For embeddings and language model (you can substitute with other providers)

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# langchain imports
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import (
    PyPDFLoader,
    PyMuPDFLoader,
    TextLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# vector stores
from langchain_community.vectorstores import Chroma

# Utilities
import numpy as np
from typing import List

e:\Babu\ragudemy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## RAG Architecture Overview
### RAG (Retrieval-Augmented Generation) Architecture:

1. Document Loading: Load documents from various sources
2. Document Splitting: Break documents into smaller chunks
3. Embedding Generation: Convert chunks into vector representations
4. Vector Storage: Store embeddings in ChromaDB
5. Query Processing: Convert user query to embedding
6. Similarity Search: Find relevant chunks from vector store
7. Context Augmentation: Combine retrieved chunks with query
8. Response Generation: LLM generates answer using context

Benefits of RAG:
- Reduces hallucinations
- Provides up-to-date information
- Allows citing sources
- Works with domain-specific knowledge

### Create sample data

In [2]:
sample_docs = [
    """
    Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    """,
    
    """
    Deep Learning and Neural Networks
    
    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning has revolutionized fields like computer vision, natural language 
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly 
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers 
    excel at sequential data processing.
    """,
    
    """
    Natural Language Processing (NLP)
    
    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.
    """
]

In [3]:
# save to file
import tempfile

temp_dir = tempfile.mkdtemp()

for i, doc in enumerate(sample_docs):
    with open(f"{temp_dir}/doc_{i}.txt", "w") as f:
        f.write(doc)

print(f"Sample document created: {temp_dir}")

Sample document created: C:\Users\LSHIVA~1\AppData\Local\Temp\tmp9cr5i63a


### Document loading

In [4]:
from langchain_community.document_loaders import DirectoryLoader

In [5]:
loader = DirectoryLoader(
    path=temp_dir,
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding':"utf-8"}
)

documents = loader.load()

print(f"Loaded {len(documents)} documents")
print(f"\nFirst document preview: ")
print(documents[0].page_content[:200] + "...")

Loaded 3 documents

First document preview: 

    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. Ther...


### Document splitting

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len,
    separators=["\n\n", "\n", ". ", " ", ""] # heirarchy of separators
)

chunks = text_splitter.split_documents(documents=documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"\nChunk example:")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

Created 7 chunks from 3 documents

Chunk example:
Content: Machine Learning Fundamentals...
Metadata: {'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}


In [7]:
chunks

[Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine Learning Fundamentals'),
 Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_1.txt'}, page_con

### Embeddings models

In [8]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [9]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### Initialize chromadb vector store and store chunks in vector representation

In [10]:
# create chromadb vector store
persist_dir = "./chroma_db"

# initialize chromadb with openai embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_dir,
    collection_name="rag_collection"
)

print(f"Vector store created with {vectorstore._collection.count()} vectors")
print(f"Persisted to: {persist_dir}")

Vector store created with 7 vectors
Persisted to: ./chroma_db


### Test similarity search

In [13]:
query = "What are the types of machine learning?"

similar_docs = vectorstore.similarity_search(query=query, k=3)
similar_docs

[Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine Learning Fundamentals'),
 Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_1.txt'}, page_content='Deep Learning and Neural Networks')]

In [14]:
print(f"Query: {query}")
print(f"\nTop {len(similar_docs)} similar chunks: ")
for i, doc in enumerate(similar_docs):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:200]+"...")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")

Query: What are the types of machine learning?

Top 3 similar chunks: 

--- Chunk 1 ---
Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine l...
Source: C:\Users\LSHIVA~1\AppData\Local\Temp\tmp9cr5i63a\doc_0.txt

--- Chunk 2 ---
Machine Learning Fundamentals...
Source: C:\Users\LSHIVA~1\AppData\Local\Temp\tmp9cr5i63a\doc_0.txt

--- Chunk 3 ---
Deep Learning and Neural Networks...
Source: C:\Users\LSHIVA~1\AppData\Local\Temp\tmp9cr5i63a\doc_1.txt


### Advanced similarity search with scores

In [15]:
results = vectorstore.similarity_search_with_score(query, k=3)
results

[(Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
  0.24275080859661102),
 (Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine Learning Fundamentals'),
  0.2676756680011749),
 (Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
  0.29335978627204895)]

#### Understanding Similarity Scores
The similarity score represents how closely related a document chunk is to your query. The scoring depends on the distance metric used:

ChromaDB default: Uses L2 distance (Euclidean distance)

- Lower scores = MORE similar (closer in vector space)
- Score of 0 = identical vectors
- Typical range: 0 to 2 (but can be higher)


Cosine similarity (if configured):

- Higher scores = MORE similar
- Range: -1 to 1 (1 being identical)

### Initialize the LLM (OpenAI), RAG chain, Prompt template, query the RAG system

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.2,
    max_tokens=500
)

In [19]:
test_response = llm.invoke("What is Large language models?")
test_response

AIMessage(content='Large language models are a type of artificial intelligence model that is trained on vast amounts of text data to generate human-like text. These models are capable of understanding and generating natural language text, and can be used for a variety of tasks such as text generation, translation, summarization, and more. Large language models have become increasingly popular in recent years due to their ability to generate high-quality text and their potential applications in various fields such as natural language processing, machine translation, and conversational AI. Some well-known examples of large language models include GPT-3 (Generative Pre-trained Transformer 3) and BERT (Bidirectional Encoder Representations from Transformers).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 13, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning

In [21]:
from langchain.chat_models.base import init_chat_model

llm = init_chat_model(
    model="openai:gpt-3.5-turbo"
)

# llm = init_chat_model(
#     model="groq:"
# )

test_response = llm.invoke("What is Large language models?")
test_response

AIMessage(content="Large language models are a type of artificial intelligence designed to process and generate human-like language at scale. These models are built using deep learning techniques and trained on massive amounts of text data to understand and generate language. They can be used for a variety of natural language processing tasks such as text generation, translation, summarization, question answering, and more. Large language models have gained popularity in recent years due to their ability to produce high-quality and coherent text outputs. Examples of large language models include OpenAI's GPT-3 and Google's BERT.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 13, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider

### Modern RAG chain

In [36]:
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [27]:
## Convert vector store to retiever
retriever = vectorstore.as_retriever(
    search_kwarg = {'k':3} ## Retrieve top 3 relevant chunks
)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001DAF62DBA10>, search_kwargs={})

In [29]:
## Create a prompt template
system_prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrived context to answer the question.
If you dont know the answer, just say that you dont know.
Use three sentences maximum and keep the answer concise.

Context: {context}
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt),
     ("human", "{input}")]
)

In [31]:
prompt.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'],
 'kwargs': {'input_variables': ['context', 'input'],
  'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are an assistant for question-answering tasks.\nUse the following pieces of retrived context to answer the question.\nIf you dont know the answer, just say that you dont know.\nUse three sentences maximum and keep the answer concise.\n\nContext: {context}\n'), additional_kwargs={}),
   HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]},
 'name': 'ChatPromptTemplate'}

### Create document chain

In [37]:
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are an assistant for question-answering tasks.\nUse the following pieces of retrived context to answer the question.\nIf you dont know the answer, just say that you dont know.\nUse three sentences maximum and keep the answer concise.\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001DAFA690190>, async_client=<openai.resou

This chain:

- Takes retrieved documents
- "Stuffs" them into the prompt's {context} placeholder
- Sends the complete prompt to the LLM
- Returns the LLM's response

#### What is create_retrieval_chain?
create_retrieval_chain is a function that combines a retriever (which fetches relevant documents) with a document chain (which processes those documents with an LLM) to create a complete RAG pipeline.

In [38]:
rag_chain = create_retrieval_chain(retriever, document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001DAF62DBA10>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are an assistant for question-answering tasks.\nUse the following pieces of retrived context to answer the question.\nIf you dont 

In [40]:
response = rag_chain.invoke({"input": "What is machine learning?"})
response

{'input': 'What is machine learning?',
 'context': [Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
  Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_0.txt'}, page_content='Machine Learning Fundamentals'),
  Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5i63a\\doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
  Document(metadata={'source': 'C:\\Users\\LSHIVA~1\\AppData\\Local\\Temp\\tmp9cr5

In [42]:
response["answer"]

'Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It includes supervised learning, unsupervised learning, and reinforcement learning as its main types. Supervised learning uses labeled data, unsupervised learning finds patterns in unlabeled data, and reinforcement learning learns through interaction with an environment.'

In [43]:
# Function to query the modern RAG system
def query_rag_modern(question):
    print(f"Question: {question}")
    print("-" * 50)
    
    # Using create_retrieval_chain approach
    result = rag_chain.invoke({"input": question})
    
    print(f"Answer: {result['answer']}")
    print("\nRetrieved Context:")
    for i, doc in enumerate(result['context']):
        print(f"\n--- Source {i+1} ---")
        print(doc.page_content[:200] + "...")
    
    return result

# Test queries
test_questions = [
    "What are the three types of machine learning?",
    "What is deep learning and how does it relate to neural networks?",
    "What are CNNs best used for?"
]

for question in test_questions:
    result = query_rag_modern(question)
    print("\n" + "="*80 + "\n")

Question: What are the three types of machine learning?
--------------------------------------------------
Answer: The three main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data for training, while unsupervised learning searches for patterns in unlabeled data. Reinforcement learning learns through a system of trial and error.

Retrieved Context:

--- Source 1 ---
Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine l...

--- Source 2 ---
Machine Learning Fundamentals...

--- Source 3 ---
Deep Learning and Neural Networks...

--- Source 4 ---
Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning...


Question